In [114]:
import cv2
import numpy as np
import os
from ultralytics import YOLO

In [115]:
model = YOLO('best.pt')

In [116]:
imgs = []

In [117]:
filenames = sorted(os.listdir('imgs_test'), key=lambda x: int(x.split('.')[0]))

In [118]:
# Load each image, convert it to a byte array, and append to the list
for filename in filenames:
    with open(f'imgs_test/{filename}', 'rb') as f:
        imgs.append(f.read())

In [119]:
# Function to convert byte data to an image
def bytes_to_image(byte_data):
    np_arr = np.frombuffer(byte_data, np.uint8)
    image = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    return image


In [120]:
# Define the entry/exit boundary line (e.g., a horizontal line at y=250)
boundary_y = 50
entry_count = 0
exit_count = 0

In [121]:
confidence_threshold = 0.9

In [122]:
previous_positions = {}

In [123]:
def process_and_track(frames):
    global entry_count, exit_count, previous_positions
    
    for byte_data in frames:
        # Convert byte data to image
        frame = bytes_to_image(byte_data)
        
        # Ensure the frame was converted successfully
        if frame is None:
            continue
        
        # Track objects in the frame
        results = model.track(source=frame)
        
        # Process the tracking results (e.g., draw bounding boxes)
        current_positions = {}
        
        for result in results:
            for box in result.boxes:
                confidence = box.conf[0].item()

                if confidence < confidence_threshold:
                    continue

                x1, y1, x2, y2 = map(int, box.xyxy.tolist()[0])
                object_id = int(box.id[0])  # Assuming 'id' contains the unique ID
                
                # Calculate the center of the bounding box
                center_y = (y1 + y2) // 2
                
                # Store the current position
                current_positions[object_id] = center_y
                
                # Draw the bounding box on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                #cv2.putText(frame, f'ID: {object_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                #cv2.putText(frame, f'{confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Check for entries and exits
        for object_id, center_y in current_positions.items():
            if object_id in previous_positions:
                previous_y = previous_positions[object_id]
                
                # Check if the object has crossed the boundary
                if previous_y <= boundary_y < center_y:
                    entry_count += 1
                elif previous_y >= boundary_y > center_y:
                    exit_count += 1
        
        # Update previous positions
        previous_positions = current_positions
        
        # Draw the boundary line
        cv2.line(frame, (0, boundary_y), (frame.shape[1], boundary_y), (0, 255, 0), 2)
        
        # Display the counts with smaller font size
        cv2.putText(frame, f'Entries: {entry_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        cv2.putText(frame, f'Exits: {exit_count}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    
        # Resize the frame to 500x500 pixels
        resized_frame = cv2.resize(frame, (500, 500))
        
        # Display the frame (optional, for visualization purposes)
        cv2.imshow('Tracked Frame', resized_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

In [124]:
process_and_track(imgs)
cv2.destroyAllWindows()


0: 640x640 1 person, 1473.1ms
Speed: 4.0ms preprocess, 1473.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 1482.0ms
Speed: 3.0ms preprocess, 1482.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1446.1ms
Speed: 3.0ms preprocess, 1446.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1410.2ms
Speed: 3.0ms preprocess, 1410.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1537.9ms
Speed: 3.0ms preprocess, 1537.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1551.3ms
Speed: 6.0ms preprocess, 1551.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 1450.1ms
Speed: 3.0ms preprocess, 1450.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1497.0ms
Speed: 3.0ms preprocess, 1497.0ms inference, 1.0ms postpr